# Middleware: Human In The Loop
<img src="./assets/LC_HITL.png" width="300">



## Setup

In [6]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****hPcA
LANGSMITH_API_KEY=****a745
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [7]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [8]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [9]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime


@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db
    
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [10]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If the database is offline, ask user to try again later without further comment.
"""

## Middleware

In [11]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
my_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro", 
    max_tokens=None,
    temperature=0 # Lower temperature for stable, factual SQL generation
)#new

agent = create_agent(
    model=my_llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    context_schema=RuntimeContext,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"execute_sql": {"allowed_decisions": ["approve", "reject"]}},
        ),
    ],
)

In [12]:
from langgraph.types import Command

question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )

    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "reject", "message": "the database is offline."}]
            }
        ),
        config=config,  # Same thread ID to resume the paused conversation
        context=RuntimeContext(db=db),
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT name FROM sqlite_master WHERE type='table';"}


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


--------------------------------------------------------------------------------
[{'type': 'text', 'text': 'The database is offline. Please try again later.', 'extras': {'signature': 'CtcJAdHtim+pFSrXvOsI2loRi+buwIYhSGGG68UkA2dCU1lQJLbwfTVDi4oieS6hexppcXCt43fAi7TQutXI0DzB6ZKufQy82wey7DjtFYGaC5iQzJBFOsco7YfU9FRHLE8FnZxy1ta4b0Mvj2Q4b5d9qi0tT0iHzE+7bohxD67rzs7Vh/K9HSc/KNKXEKQVKtI0221kkzUmtnaKcPHDrg24hdbqznw3/2SPQBEh3ZDehWpzbzsZxbom/VU0MiVlIizlHq85hIWOHNRPg7xK9vPnQfs9Zv8F1KEbOUTLlR8bKPSwdF79/Pndg104cMPDV8Z5pa6rSTvm/8wFXIiB36oQqwgId/wZhw3dirFBaYZQV9wbGEQI/iIHtuwHxePzykv8ZSjfQabNXc1XBDusUBX0IdWaTqPuCDLfB4tUEZBXQhK9oa5emMQpoRfYeEn4yN6NANtyzAojY6d1YS21bJEp1W11k1z/unlx0G7w2vUUkneZUZU2UQX38gTi9zzRjw6fHDxIIugswWjUk08r6EaHQJnW2MCakxVGn4x9WreNVwiaYe4NlzxON5VZ/V48Y7yui+YbNrNHxc1QRVkA+GgROxym5zPClDD+d/lIBH9limlDr8PxDbPl8Yk7x672gZsx+6MRi0ThNrxmfWqRJjKd1e9V0iuD0dfQpuvqaVRmHskjtgi5Oq+A/o4n2JTl1iSTVfuWUfIZLDAvxwFyp56MMtYZNIyMgICnl3fYFXf+Quv1yWRQNRp0GA9kIwe0sZTDMNyqK0FPeJBt9YVeqrUM+pVqpDRUp3EDb5ler8TY0ORx

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [13]:
config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

while "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )
    
    result = agent.invoke(
        Command(
            resume={"decisions": [{"type": "approve"}]}
        ),
        config=config,  # Same thread ID to resume the paused conversation
        context=RuntimeContext(db=db),
    )

for msg in result["messages"]:
    msg.pretty_print()

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2, model: gemini-2.5-pro
Please retry in 57.42760663s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelangu

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT FirstName, LastName FROM employees LIMIT 5'}


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2, model: gemini-2.5-pro
Please retry in 55.382156905s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelang

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT name FROM sqlite_master WHERE type='table'"}


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT FirstName, LastName FROM Employee'}


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================ Human Message =================================

What are the names of all the employees?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (ec887eab-2311-4354-b9f9-3f3b9c157b64)
 Call ID: ec887eab-2311-4354-b9f9-3f3b9c157b64
  Args:
    query: SELECT FirstName, LastName FROM employees LIMIT 5
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: employees
[SQL: SELECT FirstName, LastName FROM employees LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================

[{'type': 'text', 'text': "I can't find a table named `employees`. Let me see what tables are in the database.", 'extras': {'signature': 'CuYCAdHtim/EhHyW1NjlG/OcE+ogxBNqEUy6n16Qkbx/gzf270/ZwmwbRuFNLLtieOy4lXUFgheocqXEoE2/HZ/3VLmuxZbAeFvf4NKnwfCBEyjC/5xJmx

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
